In [1]:
import os
import pandas as pd
import numpy as np
import json

In [2]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [3]:
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = CONFIG_DATA['NUMBER_OF_FEATURES']

CLASSIFICATIONS = CONFIG_DATA['CLASSES_MOTIONSPEED']

LABEL_COLUMN = 'Class_MotionSpeed'

# processed data folder path
DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\TRAIN2\\'

# save numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'
ALL_Y_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'

In [4]:
LABEL_TO_CATEGORY = {label: category for category, label in enumerate(CLASSIFICATIONS)}

print(LABEL_TO_CATEGORY)

{'SLOW': 0, 'AVERAGE': 1, 'FAST': 2}


In [5]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and "SML" in file and "STEPS" in file and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TRAIN2-SIDESTEPS-L-SML-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-60BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-80BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-SML-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-SML-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-SML-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-SML-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-SML-60BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-SML-80BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-110BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-55BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-60BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-SML-80BPM.xlsx']


In [6]:
def normalizeSensorData(sensor_input):
    return (sensor_input + 180)/360


In [7]:
def df2Xy(df, windowSize=5):
  X = []
  y = []

  # NORMALIZE THE SENSOR DATA
  #df["L_Pitch_Delta"] = df["L_Pitch_Delta"].apply(normalizeSensorData)
  #df["L_Roll_Delta"] = df["L_Roll_Delta"].apply(normalizeSensorData)
  #df["R_Pitch_Delta"] = df["R_Pitch_Delta"].apply(normalizeSensorData)
  #df["R_Roll_Delta"] = df["R_Roll_Delta"].apply(normalizeSensorData)

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input_data_list = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values = df.loc[i + j, ['L_Pitch_Delta', 'L_Roll_Delta', 'R_Pitch_Delta', 'R_Roll_Delta']].values.tolist()

        # add row values to the input
        input_data_list.append(row_values)

    # turnn list into array to do arthimetic
    input_data_array = np.array(input_data_list)

    # remove all negative sinces we want to the total change (we dont care which direction)
    input_data_array = np.abs(input_data_array)

    # calculate the total change for each sennsor value (TOTAL POSITIVE SENSOR CHANGES)
    input_data_array = np.sum(input_data_array, axis=0)

    if(np.isnan(input_data_array).any() == False):
      # turn back to list
      input_data_list = input_data_array.tolist()

      # add our input to our total inputs, marked as X
      X.append(input_data_list)

      # outputs: y labels
      label = df.loc[i + (windowSize - 1), [LABEL_COLUMN]].values.tolist()

      y.append(label)

  return (np.array(X), np.array(y))

In [8]:
ALL_X_TRAIN = np.empty((0, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN = np.empty((0))  # List to store all Y training data

In [9]:
def processData(ALL_X, ALL_Y):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        xTrain, yTrain = df2Xy(df, WINDOW_SIZE)

        yTrain = yTrain.reshape(-1)
        
        ALL_X = np.concatenate([ALL_X, xTrain])
        ALL_Y = np.concatenate([ALL_Y, yTrain])
        
        print('xTrain.shape:', xTrain.shape)
        print('ALL_X_TRAIN.shape:', ALL_X.shape, ' ALL_Y_TRAIN.shape:', ALL_Y.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_label = np.vectorize(LABEL_TO_CATEGORY.get)(ALL_Y)

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TRAIN_CLASSIFY_PATH, ALL_X)
    np.save(ALL_Y_TRAIN_CLASSIFY_PATH, numerical_label)

In [10]:
if not os.path.exists(ALL_X_TRAIN_CLASSIFY_PATH):
    processData(ALL_X_TRAIN, ALL_Y_TRAIN)

ALL_X_TRAIN_CLASSIFY = np.load(ALL_X_TRAIN_CLASSIFY_PATH)
ALL_Y_TRAIN_CLASSIFY = np.load(ALL_Y_TRAIN_CLASSIFY_PATH)

print(ALL_X_TRAIN_CLASSIFY.shape)
print(ALL_Y_TRAIN_CLASSIFY.shape)

xTrain.shape: (3389, 4)
ALL_X_TRAIN.shape: (3389, 4)  ALL_Y_TRAIN.shape: (3389,)
-------------------------------
xTrain.shape: (3060, 4)
ALL_X_TRAIN.shape: (6449, 4)  ALL_Y_TRAIN.shape: (6449,)
-------------------------------
xTrain.shape: (6798, 4)
ALL_X_TRAIN.shape: (13247, 4)  ALL_Y_TRAIN.shape: (13247,)
-------------------------------
xTrain.shape: (6140, 4)
ALL_X_TRAIN.shape: (19387, 4)  ALL_Y_TRAIN.shape: (19387,)
-------------------------------
xTrain.shape: (3390, 4)
ALL_X_TRAIN.shape: (22777, 4)  ALL_Y_TRAIN.shape: (22777,)
-------------------------------
xTrain.shape: (3061, 4)
ALL_X_TRAIN.shape: (25838, 4)  ALL_Y_TRAIN.shape: (25838,)
-------------------------------
xTrain.shape: (4003, 4)
ALL_X_TRAIN.shape: (29841, 4)  ALL_Y_TRAIN.shape: (29841,)
-------------------------------
xTrain.shape: (3699, 4)
ALL_X_TRAIN.shape: (33540, 4)  ALL_Y_TRAIN.shape: (33540,)
-------------------------------
xTrain.shape: (8026, 4)
ALL_X_TRAIN.shape: (41566, 4)  ALL_Y_TRAIN.shape: (41566,)
-

In [11]:
print(np.max(ALL_X_TRAIN_CLASSIFY))

121.0
